In [1]:
import os
import sys
import gc
# assuming data, models, engine in flicc directory:
flicc_path = os.path.join(os.path.dirname(os.getcwd()), '')
sys.path.append(flicc_path)
import torch
from data import ClimateDataset
from models import ClassificationModel
from engine import Engine

In [2]:
model_checkpoint='bert-base-uncased'

In [3]:
g = 4
lr = 1.0e-4
wd = 0.1

data = ClimateDataset(model_to_train=1,model_checkpoint=model_checkpoint,dataset_url=flicc_path,batch_size=32)
data.setup_dataloaders()
model = ClassificationModel(model_checkpoint=data.model_checkpoint,num_labels=data.num_labels)
trainer = Engine(epochs=30,labels=data.labels)
trainer.model = model.model
trainer.dataset_encoded = data.dataset_encoded
test_acc, test_f1, eval_acc, eval_f1 = trainer.run(lr=lr,
                                                    wd=wd,
                                                    train_dataloader=data.train_dataloader,
                                                    eval_dataloader=data.eval_dataloader,
                                                    test_dataloader=data.test_dataloader,
                                                    focalloss=True,
                                                    gamma=g,
                                                    early_stop=3)

print(f'test_acc: {test_acc:.2f}, test_f1: {test_f1:.2f}, eval_acc: {eval_acc:.2f}, eval_f1: {eval_f1:.2f}')
trainer.save_best_model()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 30: Train Loss:	0.0853	Val Loss:	0.6082	Accuracy:	0.7484	F1:	0.7161
2 / 30: Train Loss:	0.0600	Val Loss:	0.4755	Accuracy:	0.8359	F1:	0.7608
3 / 30: Train Loss:	0.0449	Val Loss:	0.4224	Accuracy:	0.8206	F1:	0.7645
4 / 30: Train Loss:	0.0340	Val Loss:	0.5048	Accuracy:	0.8578	F1:	0.8023
5 / 30: Train Loss:	0.0190	Val Loss:	0.5972	Accuracy:	0.6433	F1:	0.6309
6 / 30: Train Loss:	0.0140	Val Loss:	0.4132	Accuracy:	0.8249	F1:	0.7740
7 / 30: Train Loss:	0.0138	Val Loss:	0.4277	Accuracy:	0.8118	F1:	0.7324
No improvement for 3 epochs. Stopping early.
best (higgest macro f1-score) val results:
              precision    recall  f1-score   support

       bknow       0.78      0.63      0.70       119
      struct       0.88      0.94      0.91       338

    accuracy                           0.86       457
   macro avg       0.83      0.78      0.80       457
weighted avg       0.85      0.86      0.85       457

test results:
              precision    recall  f1-score   support

       bknow